# Gradient Boosting Regression Pipeline

This notebook walks through a full machine learning pipeline for predicting the `RANGE` variable using **Gradient Boosting Regressor**. The pipeline includes:
- Loading and preparing the dataset
- Feature scaling
- Hyperparameter tuning using Randomized Search with Cross-Validation
- Evaluating model performance with metrics such as MSE, RMSE, MAE, and R²
- Analyzing feature importance
- Visualizing residuals and saving the final model for future use

## Load and Inspect Dataset

We begin by loading the preprocessed dataset using `pandas.read_csv`. This dataset contains features and a target variable named `RANGE`. After loading, we print the shape of the dataset to confirm its structure.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

# Load and prepare data
print("Loading data...")
data = pd.read_csv('../data/processed/aggregated_dataset.csv')
print("Dataset shape:", data.shape)

## Separate Features and Target

We drop the `RANGE` (target) and `NLOS` (non-line-of-sight flag) columns from the dataset to form the feature matrix `X`. The `RANGE` column is stored separately as the target variable `y`.

We also split the data into training and test sets using `train_test_split`, with 80% used for training and 20% for testing.


In [ ]:
# Separate features and target
X = data.drop(['RANGE', 'NLOS'], axis=1)  # Remove both RANGE and NLOS
y = data['RANGE']  # Target is RANGE

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Scale Features with RobustScaler

We use `RobustScaler` to scale the input features. This scaler is robust to outliers, as it uses the median and interquartile range rather than mean and standard deviation. We fit it to the training data and apply the same transformation to the test data.


In [ ]:
# Scale the features
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Initialize Gradient Boosting Regressor and Define Hyperparameter Space

We initialize a `GradientBoostingRegressor` with a fixed `random_state` for reproducibility.

Then, we define a dictionary of hyperparameters (`param_distributions`) to explore. In this case, we're specifying fixed values for each key hyperparameter to evaluate via randomized search.


In [ ]:
# Initialize base model
rf_reg = GradientBoostingRegressor(random_state=42)


# Random search for hyperparameter tuning
rf_random = RandomizedSearchCV(
    estimator=rf_reg,
    # Define hyperparameter search space
    param_distributions={
    'n_estimators': [100],
    'max_depth': [20],
    'min_samples_split': [5],
    'min_samples_leaf': [4],
    'max_features': ['sqrt']
},
    n_iter=1,
    cv=4,
    random_state=42,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    verbose=2
)

## Train the Model on Training Data

The model is now fit to the training data (`X_train_scaled`, `y_train`). The best hyperparameters and the corresponding MSE score from cross-validation are printed after training.


In [ ]:
# Fit the random search model
rf_random.fit(X_train_scaled, y_train)

# Print best parameters and score
print("\nBest parameters:", rf_random.best_params_)
print("Best MSE score:", -rf_random.best_score_)

## Retrieve Best Estimator

Once training and tuning are complete, we extract the best-performing model (based on CV MSE score) for further evaluation and predictions.


In [ ]:
# Get the best model
best_rf_reg = rf_random.best_estimator_

# Make predictions
y_pred = best_rf_reg.predict(X_test_scaled)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nTest Set Performance:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


In [ ]:
# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Range')
plt.ylabel('Predicted Range')
plt.title('Actual vs Predicted Range Values')
plt.tight_layout()
plt.show()

## Visualize Top 20 Feature Importances

We extract and plot the top 20 most important features according to the trained Gradient Boosting model. This helps interpret which input variables have the most influence on predictions.


In [ ]:
# Plot feature importances
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': best_rf_reg.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False).head(20)

plt.figure(figsize=(12, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Top 20 Most Important Features for Range Prediction')
plt.tight_layout()
plt.show()

## Save Trained Model and Scaler

The trained model (`best_rf_reg`) and the `RobustScaler` used for feature scaling are saved using `joblib`. This allows the model to be reused later without retraining.


In [ ]:
# Save the model and scaler
joblib.dump(best_rf_reg, '../models/best_rf_regressor.joblib')
joblib.dump(scaler, '../models/rf_regressor_scaler.joblib')

## Residual Analysis

Residuals (actual - predicted values) give insight into the prediction errors. A scatter plot of residuals helps identify patterns such as bias or non-random error distributions.


In [ ]:
# Calculate residuals
residuals = y_test - y_pred

# Plot residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Range')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.tight_layout()
plt.show()

### Plot Distribution of Residuals

A histogram shows the distribution of residuals. Ideally, residuals should be roughly normally distributed around 0, indicating unbiased predictions.


In [ ]:
# Plot residuals distribution
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50)
plt.xlabel('Residual Value')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.tight_layout()
plt.show()

### Summary Statistics of Residuals

We display descriptive statistics (mean, std, quartiles) for the residuals to better understand the spread and central tendency of prediction errors.


In [ ]:
# Print summary statistics of residuals
print("\nResiduals Summary Statistics:")
print(pd.Series(residuals).describe())